In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;


In [0]:
bronze_df = spark.read.table("workspace.bronze.population_raw")

In [0]:
from pyspark.sql.functions import col, trim
from pyspark.sql.types import IntegerType, LongType, DoubleType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# -----------------------------
# 1. Read Bronze
# -----------------------------
bronze_df = spark.read.table("workspace.bronze.population_raw")

# -----------------------------
# 2. Clean & type
# -----------------------------
clean_df = (
    bronze_df
    .withColumn("state_code", trim(col("state_code")))
    .withColumn("rank_2014", col("rank_2014").cast(IntegerType()))
    .withColumn("population_estimate_2014", col("population_estimate_2014").cast(LongType()))
    .withColumn("median_sales_price_2015", col("median_sales_price_2015").cast(DoubleType()))
    .dropna(subset=["state_code"])
)

# -----------------------------
# 3. Deduplicate by MERGE key
# -----------------------------
window_spec = Window.partitionBy("state_code").orderBy(col("ingestion_timestamp").desc())

silver_updates_df = (
    clean_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)

silver_updates_df.createOrReplaceTempView("silver_updates")


In [0]:
%sql
MERGE INTO workspace.silver.population_clean AS target
USING silver_updates AS source
ON target.state_code = source.state_code

WHEN MATCHED THEN
  UPDATE SET
    target.rank_2014 = source.rank_2014,
    target.population_estimate_2014 = source.population_estimate_2014,
    target.median_sales_price_2015 = source.median_sales_price_2015

WHEN NOT MATCHED THEN
  INSERT (
    state_code,
    rank_2014,
    population_estimate_2014,
    median_sales_price_2015
  )
  VALUES (
    source.state_code,
    source.rank_2014,
    source.population_estimate_2014,
    source.median_sales_price_2015
  );


In [0]:
%sql
SELECT * 
FROM workspace.silver.population_clean
LIMIT 10;